In [ ]:
import math
import torchinfo
import os
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
import copy



from torch import nn
from System.pos_enc import *
from torch.optim.lr_scheduler import StepLR
from Data.MowingDetectionDataset_Parcelized import *
from System.pos_enc import *
from System.Engine import *
from System.Engine_Tr import *
from helpers.various import *
from torchmetrics import Accuracy, ConfusionMatrix, F1Score, Recall, Precision
from torchinfo import summary


transforms = A.Compose([
        #A.Normalize(mean=[-0.3357, -0.3259],#####how many channels to include?
                   # std=[0.4688, 0.4685]),
        ToTensorV2()
    ])

In [ ]:
###padding is used because there is the possibility to have nan at the ts start###
###also the argument nan_free is passed to interpolate the ts###
train_dt = ParcelMowingDataset(root_path = root_dataset_path,labels_path = path_train, trsm = transforms,max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
train_dloader = DataLoader(train_dt, sampler=ImbalancedDatasetSampler(train_dt), 
                           batch_size=train_batch, shuffle=False, num_workers=0)

val_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_val, trsm = transforms, max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
val_dloader = DataLoader(val_dt, batch_size=val_batch, shuffle=False, num_workers=0)

test_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_test, trsm = transforms, max_lenght_to_pad=48,
                nan_free = False,mode = 'pad', interpolate = True,extra_features = True)
test_dloader = DataLoader(test_dt, batch_size=test_batch, shuffle=False, num_workers=0)

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.tuner.tuning import Tuner


callbacks = [
    EarlyStopping(monitor="accuracy_macro/val", mode="max", patience=50),
    ModelCheckpoint(monitor="accuracy_macro/val", mode="max", save_last=True),
    LearningRateMonitor(logging_interval='epoch'),
]


In [ ]:
from torchinfo import summary
model = resnet.load_from_checkpoint('./epoch=66-step=134.ckpt')

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False
# Get the number of input features for the last FC layer

num_features = model.resnet1d.fc.in_features
print('num features_in',num_features)
# Define your new FC layer
# For example, if you want to replace the FC layer with a new one for a different classification task:
new_fc = nn.Linear(num_features, 5)  # num_classes is the number of classes in your new task

# Replace the FC layer in the ResNet model
model.resnet1d.fc = new_fc


# Print trainable parameters per layer
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Trainable parameters: {param.numel()}")

        
        
summary(model, input_size=(1, 1, 48, 5))

In [ ]:
test_model = Pretrained_ECGTR()

In [ ]:
from torchinfo import summary
summary(test_model, input_size=(1, 1, 48, 5))
# Print trainable parameters per layer
for name, param in test_model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Trainable parameters: {param.numel()}")

In [ ]:
###resume from checkpoint###
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.tuner.tuning import Tuner


callbacks = [
    EarlyStopping(monitor="accuracy_macro/val", mode="max", patience=50),
    ModelCheckpoint(monitor="accuracy_macro/val", mode="max", save_last=True),
    LearningRateMonitor(logging_interval='epoch'),
]

model = ECGTRMODEL()
trainer = pl.Trainer(
    accelerator="gpu", 
    devices=1,
    max_epochs=number_of_epochs,
    callbacks=callbacks,
    default_root_dir=save_dir,
    enable_progress_bar = False,
    resume_from_checkpoint = './pretrained_models_resumables/lightning_logs/version_2/checkpoints/epoch=37-step=2774.ckpt'
    #detect_anomaly=True,
)

trainer.fit(model, train_dataloaders=train_dloader, val_dataloaders=val_dloader)

In [ ]:
###finetune a pretrained model###


from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.tuner.tuning import Tuner


callbacks = [
    EarlyStopping(monitor="accuracy_macro/val", mode="max", patience=50),
    ModelCheckpoint(monitor="accuracy_macro/val", mode="max", save_last=True),
    LearningRateMonitor(logging_interval='epoch'),
]

model = Pretrained_ECGTR()
trainer = pl.Trainer(
    accelerator="gpu", 
    devices=1,
    max_epochs=number_of_epochs,
    callbacks=callbacks,
    default_root_dir=save_dir,
    enable_progress_bar = False,

)

trainer.fit(model, train_dataloaders=train_dloader, val_dataloaders=val_dloader)

In [ ]:
best_model = Pretrained_ECGTR.load_from_checkpoint('./___.ckpt')
trainer = pl.Trainer(accelerator="gpu", 
    devices=1)
trainer.test(best_model, dataloaders=test_dloader)

In [ ]:
#####to do!!!!####
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt


class_labels = ['0 times', '1 times','2 times', '3 times', '4 times']
cm = best_model.test_confusion_matrix.compute().cpu().numpy()

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=class_labels )
plt.figure(figsize=(5,5), dpi=100)
ax = plt.axes()

disp.plot(ax=ax)
plt.xticks(rotation=90)
plt.show()